### Some analysis

In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os
import re

In [2]:
keys = {
    'temperature.pv',
    'agitation.pv',
    'ph.outputDown',
    'ph.manDown',
    'do.manUp',
    'do.outputDown',
    'do.pv',
    'do.manDown',
    'do.outputUp',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'level.pv',
    'Elapsed',
    'Elapsed.hr',
    'Time'
}
def hide_columns(ws):
    for col in ws.UsedRange.Columns:
        c = col.Cells(1,1)
        if c.Value not in keys:
            col.Hidden=True

In [3]:
def data_range(ws, name, begin_offset=0):
    c1 = ws.Cells.Find(name)
    if c1 is None: raise ValueError(name)
    c1 = c1.Offset(2+begin_offset,1)
    c2 = c1.End(xlc.xlDown)
    return c1, c2

class SeriesProxy():
    def __init__(self, ws, name, xname="Elapsed", yax=1, begin_offset=0):
        self.ws = ws
        self.name = name
        self.yax = yax
        self.s = None
        self.xname = xname
        self.begin_offset=begin_offset
        
    def create(self, chart):
        e1, e2 = data_range(self.ws, self.xname, self.begin_offset)
        c1, c2 = data_range(self.ws, self.name, self.begin_offset)
        cr = self.ws.Cells.Range
        self.xr = cr(e1, e2)
        self.yr = cr(c1, c2)
        self.s = CreateDataSeries(chart, self.xr, self.yr, self.name)
        
    def set_name(self, n):
        self.s.Name = n 
        
class ChartProxy():
    def __init__(self, ws, chart=None, *args):
        self.ws = ws
        self.chart = chart or CreateChart(ws, *args)
        self.chart.ChartStyle = 240  # Excel 2017 style
        self.series = []
        
    def make_series(self, series):
        for s in series:
            s.create(self.chart)
            self.series.append(s)
            
    def _axlabel(self, ax, t):
        ax.HasTitle=True
        ax.AxisTitle.Text=t
    
    def xlabel(self, t):
        xax = self.chart.Axes(xlc.xlCategory, xlc.xlPrimary)
        self._axlabel(xax, t)
        
    def ylabel(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlPrimary)
        self._axlabel(yax, t)
        
    def ylabel2(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlSecondary)
        self._axlabel(yax, t)
        
    def title(self, t):
        self.chart.HasTitle = True
        self.chart.ChartTitle.Text = t
        
    def copyto(self, r):
        # chart parent -> ChartObject
        self.chart.Parent.Copy()
        r.Paste()
         
def _listify(args):
    l = []
    if isinstance(args, str):
        return [args]
    for a in args:
        if isinstance(a, (list,tuple)):
            l.extend(_listify(a))
        else:
            l.append(a)
    return l
    
def make_chart(names, title=None, xlabel=None, ylabel=None, xname="Elapsed", *, uws=None, begin_offset=0):
    uws = uws or ws
    c = ChartProxy(uws)
        
    names = _listify(names)
    c.make_series(SeriesProxy(uws,n, xname, 1, begin_offset) for n in names)
    
    # optional setup
    if title: c.title(title)
    if xlabel: c.xlabel(xlabel)
    if ylabel: c.ylabel(ylabel)
    
    return c

### Example code for now to use functions

In [4]:
# # MFCs
# mfcc = [
#     "MFCs.Air",
#     "MFCs.O2",
#     "MFCs.N2",
#     "MFCs.CO2"
# ]
# make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
# make_chart("do.pv", "DO PV", "Time(s)", "DO PV (%)")

#### Some analysis

In [36]:
def column_data(name):
    c1 = ws.Cells.Find(name)
    c2 = c1.End(xlc.xlDown)
    data = ws.Cells.Range(c1.Offset(2,1),c2).Value
    return [d[0] for d in data]
    
def add_af1(name, new, form):
    e1 = ws.Cells.Find(name)
    e1.Offset(1,2).EntireColumn.Insert()
    e1.Offset(1,2).Value = new
    
    c1 = e1.Offset(2,2)
    c2 = e1.End(xlc.xlDown).Offset(1,2)
    c1.Value = form % e1.Offset(2,1).GetAddress(0,0)
    afr = ws.Cells.Range(c1, c2)
    c1.AutoFill(afr)
    return c1, c2
    

def nch(ws, n):
    c = CreateChart(ws)
    c.Location(xlc.xlLocationAsNewSheet, n)
    return ws.Parent.Charts(n)

def gfp(f): return os.path.abspath(f)


def sorteddir():
    return sorted((f for f in os.listdir() if os.path.isfile(f)))

def trendclr(c):
    ns = c.SeriesCollection().Count
    l = c.Legend.LegendEntries
    for i in range(2*ns, ns, -1):
        l(i).Delete()
        
def mktrend(c):
    AddTrendlines(c)
    trendclr(c)
    
    
def anylf(wb,ws,name):
    e1, e2 = add_af1("Elapsed", "Elapsed.hr", "=%s/3600")
    ws.Cells.Range(e1, e2).NumberFormat = "0.000"

    c1 = make_chart("do.pv", name, "Time(hr)", 
                   "DO PV (%)", "Elapsed.hr", uws=ws)
    # MFCs
    mfcc = [
        "MFCs.Air",
        "MFCs.O2",
        "MFCs.N2",
        "MFCs.CO2"
    ]
    c2 = make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
    
    return c1, c2
    
def analyze(files):
    # globals make debugging easy
    global wb, ws, nwb, ws1, ws2
    xl.DisplayAlerts=False
    
    nwb = xl.Workbooks.Add()
    ws1 = nwb.Worksheets(1)
    ws2 = nwb.Worksheets.Add()  # new workbook only comes with 1 ....
    n1 = n2 = 0
    
    for f in files:
        print("Analyzing", f)
        _, ppii, st_sp, ctrls, idn = f.split()
        idn = idn.split(".")[0].split("_")[1]
        start, _, sp = st_sp.split("_")
        ag, mg, temp = ctrls.split("_")
        
        
        wb = xl.Workbooks.Open(gfp(f))
        ws = wb.Worksheets(1)
        
        name = "%s to %s %s LPM" % (start, sp, mg)
        c1, c2 = anylf(wb,ws,name)
        
        if mg == "0.1":
            c1.copyto(ws1)
            nc1 = ws1.ChartObjects(n1*2+1)
            nc1.Left = 12
            nc1.Height = 300
            nc1.Width = 510
            nc1.Top  = 10 + 310 * n1
            
            c2.copyto(ws1)
            nc2 = ws1.ChartObjects(n1*2+2)
            nc2.Left = 550
            nc2.Height = 300
            nc2.Width = 510
            nc2.Top = 10 + 310 * n1
            
            n1 += 1
        else:
            c1.copyto(ws2)
            nc1 = ws2.ChartObjects(n2*2+1)
            nc1.Left = 12
            nc1.Height = 300
            nc1.Width = 510
            nc1.Top  = 10 + 310 * n2
            
            c2.copyto(ws2)
            nc2 = ws2.ChartObjects(n2*2+2)
            nc2.Left = 550
            nc2.Height = 300
            nc2.Width = 510
            nc2.Top = 10 + 310 * n2
            
            n2 += 1
            
        wb.SaveAs(gfp(".\\analyzed\\"+f.replace(".csv", ".xlsx")), FileFormat=xlc.xlOpenXMLWorkbook)
        wb.Close(True)
        
    print("Finished analysis!")

In [37]:
fp = 'C:\\Users\\natha\\Documents\\test\\do_3L_verif_ms_diptube 170920'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed", exist_ok=True)
files = sorteddir()
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze(files)

Analyzing 3L_do_pid 4_-20_55_40 100_to_150 32_0.1_37 id_21.1.csv
Analyzing 3L_do_pid 4_-20_55_40 100_to_150 32_0.5_37 id_22.1.csv
Analyzing 3L_do_pid 4_-20_55_40 100_to_50 32_0.1_37 id_21.3.csv
Analyzing 3L_do_pid 4_-20_55_40 100_to_50 32_0.5_37 id_22.3.csv
Analyzing 3L_do_pid 4_-20_55_40 150_to_100 32_0.1_37 id_21.2.csv
Analyzing 3L_do_pid 4_-20_55_40 150_to_100 32_0.5_37 id_22.2.csv
Finished analysis!


In [33]:
s = xl.Selection
s.Top, s.Width, s.Height, s.Left

(7.0, 506.3333070866142, 298.8, 11.399921417236328)